In [1]:
!pip install autogluon.tabular
!pip install ray==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: ray
    Found existing installation: ray 2.40.0
    Uninstalling ray-2.40.0:
      Successfully uninstalled ray-2.40.0


In [2]:
import numpy as np
import pandas as pd

In [3]:
sub = pd.read_csv("/kaggle/input/playground-series-s4e11/sample_submission.csv")
original = pd.read_csv("/kaggle/input/depression-surveydataset-for-analysis/final_depression_dataset_1.csv")
train = pd.read_csv("/kaggle/input/playground-series-s4e11/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e11/test.csv")

In [4]:
train = train.drop('id', axis = 1)
test_ids = test['id'].values
test = test.drop('id', axis = 1)
original["Depression"] = original["Depression"].map({"Yes": 1, "No": 0})
train = pd.concat([train, original], ignore_index=True)

In [5]:
train.columns

Index(['Name', 'Gender', 'Age', 'City', 'Working Professional or Student',
       'Profession', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration',
       'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress',
       'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [6]:
original.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Pooja,Female,37,Ghaziabad,Working Professional,Teacher,NaN,2.0,NaN,NaN,4.0,7-8 hours,Moderate,MA,No,6,2,No,0
1,Reyansh,Male,60,Kalyan,Working Professional,Financial Analyst,NaN,4.0,NaN,NaN,3.0,5-6 hours,Unhealthy,B.Com,Yes,0,4,Yes,0
2,Manvi,Female,42,Bhopal,Working Professional,Teacher,NaN,2.0,NaN,NaN,3.0,5-6 hours,Moderate,M.Com,No,0,2,No,0
3,Isha,Female,44,Thane,Working Professional,Teacher,NaN,3.0,NaN,NaN,5.0,7-8 hours,Healthy,MD,Yes,1,2,Yes,0
4,Aarav,Male,48,Indore,Working Professional,UX/UI Designer,NaN,4.0,NaN,NaN,3.0,7-8 hours,Moderate,BE,Yes,6,5,Yes,0


In [7]:
for col in ["Name", "City", "Profession", "Degree",  "Dietary Habits", "Sleep Duration"]:
    unique_vals = original[col].unique()
    train.loc[~train[col].isin(unique_vals), col] = "FAKE"
    test.loc[~test[col].isin(unique_vals), col] = "FAKE"
cat_c = test.select_dtypes(include="object").columns

train[cat_c] = train[cat_c].fillna('None').astype("category")
test[cat_c] = test[cat_c].fillna('None').astype("category")

In [8]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(
    label="Depression",
    eval_metric="roc_auc",
    problem_type="binary",
    verbosity=1,
)

predictor.fit(
    train_data=train,
    time_limit=(3600*11),
    presets="best_quality",
)

predictor.fit_summary(verbosity=1)
predictions = predictor.predict(test)

No path specified. Models will be saved in: "AutogluonModels/ag-20250119_012845"
2025-01-19 01:28:48,570	INFO worker.py:1752 -- Started a local Ray instance.


*** Summary of fit() ***
Estimated performance of each model:
                          model  score_val eval_metric  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           WeightedEnsemble_L3   0.976614     roc_auc     272.622149  21710.153773                0.025909          44.247659            3       True         63
1           WeightedEnsemble_L2   0.976591     roc_auc      73.885781  11488.305937                0.025776          25.519042            2       True         35
2           CatBoost_r13_BAG_L2   0.976509     roc_auc     270.171890  20445.988297                0.664387         853.724741            2       True         58
3               CatBoost_BAG_L2   0.976468     roc_auc     270.076723  19924.285663                0.569220         332.022107            2       True         40
4          CatBoost_r137_BAG_L2   0.976460     roc_auc     270.071806  19999.527267                0.564302         407.263710  

In [9]:
df = pd.DataFrame({'id': test_ids, 'Depression': predictions})
df.to_csv("submission.csv", index=False)